# Gerar arquivo CSV
Essa seção diz respeito ao procedimento de pegar todos os arquivos *DBC* e gerar apenas um único arquivo *CSV*, a fim de facilitar a manipulação dos dados posteriormente.

## Sumário
- [Dependências](#dependencias)
  - [Instalação](#instalacao)
  - [Importação](#importacao)
- [Programação](#programação)
  - [Filtragem](#filtragem)
    - [Por ano](#por_ano)
    - [Por CIDs](#por_cid)
  
## <a id="dependencias">Dependências</a>
### <a id="instalacao">Instalação</a>
-  **danicat/read.dbc**: é um pacote que importa dados de um *DBC* (compressão de *DBF*) para data frames.

In [ ]:
devtools::install_github("danicat/read.dbc")

### <a id="importacao">Importação</a>
-  **read.dbc**: usamos para ler os arquivos *DBC*;
-  **plyr**: usamos a função *rbind.fill* para juntar dataframes de diferentes tamanhos. Ele inclui, por padrão, NA como valor nas colunas inseridas entre um dataframe e outro.

In [1]:
library('read.dbc')
library(plyr)
library(readr)

### <a id="programacao">Programação</a>
Nessa seção, nossa intenção é filtrar os dados presentes nos *DBC* para apenas os que iremos utilizar, que são:

| Coluna | Descrição     |
| ------:|:------------- |
| DTOBITO | Data do óbito (dd mm aaaa) |
| DTNASC | Data do nascimento (dd mm aaaa) |
| SEXO | Sexo |
| RACACOR | Raça |
| ESTCIV | Situação conjugal |
| OCUP | Ocupação habitual e ramo de atividade |
| CODMUNRES | Código do município de residência |
| LOCOCOR | Local de ocorrência do óbito |
| CODMUNOCOR | Código do município de ocorrência |
| IDADEMAE | Idade da mãe |
| ESCMAE | Escolaridade em anos da mãe |
| OCUPMAE | Ocupação da mãe |
| QTDFILVIVO | Número de filhos vivos |
| QTDFILMORT | Número de filhos mortos |
| GRAVIDEZ | Informa o tipo de gravidez |
| GESTACAO | Faixa de semanas da gestação |
| CAUSABAS | Causa básica da DO |
| LINHAA | CIDs informados na Linha A da DO |
| LINHAB | CIDs informados na Linha B da DO |
| LINHAC | CIDs informados na Linha C da DO |
| LINHAD | CIDs informados na Linha D da DO |
| LINHAII | CIDs informados na Linha II da DO |
| CIRCOBITO | Indicar qual foi a provável circunstância de morte não natural |
| FONTE | Indicar a fonte da informação, conforme a tabela |

In [2]:
# Contém o caminho até a pasta que contêm os arquivos gerais que iremos usar
files_path <- paste(getwd(), 'files', sep='/')
# Contém o caminho até a pasta que contêm os arquivos DBC que serão tratados
dores_path <- paste(files_path, 'dores', sep='/')
# Contém o caminho até a pasta que contêm os arquivos CSV com os registros de cada ano
anos_path <- paste(files_path, 'ano', sep='/')
# Contém o caminho até a pasta que contêm os arquivos CSV finais
data_path <- paste(getwd(), 'data', sep='/')
# Contém um vetor com as colunas que queremos filtrar
cols_name <- c("DTOBITO", "HORAOBITO", "DTNASC", "IDADE", "SEXO", "RACACOR", "ESTCIV", "OCUP",
               "CODMUNRES", "LOCOCOR", "CODMUNOCOR", "IDADEMAE", "ESCMAE", "OCUPMAE", "QTDFILVIVO",
               "QTDFILMORT", "LINHAA", "LINHAB", "LINHAC", "LINHAD", "LINHAII", "ESC2010", "ESCMAE2010")

### <a id="filtragem">Filtragem</a>
Aqui iremos filtrar os arquivos *DBC*, de forma que, ao final, iremos gerar um arquivo *CSV* para cada ano (2011 a 2015) e, após, um arquivo com os dados de suicídios que ocorreram nesses mesmos anos.

#### <a id="por_ano">Por ano</a>
Primeiro iremos filtrar os arquivos *DBC* pelo ano. Ao final, iremos gerar um arquivo *CSV* para cada ano.

In [7]:
for (ano in 2011:2015) {
    ano <- paste(ano, '', sep='')
    filestrsave <- paste(anos_path, paste(ano, '.csv', sep=''), sep='/')
    
    # Listando os arquivos
    listoffiles <- list.files(dores_path, full.names=TRUE, pattern=ano)

    # Contador e barra de progresso, respectivamente
    i_count <- 0
    pb <- txtProgressBar(min=0, max=length(listoffiles), style=3)

    # Abrindo os arquivos
    mydata <- lapply(listoffiles, function (file) {
        # Pegamos o arquivo, transformamos ele em list
        data_dbc <- read.dbc(file)
        # Avançamos no contador, ou seja, na barra de progresso
        i_count <<- i_count + 1
        setTxtProgressBar(pb, i_count)

        return(data_dbc)
    })
    
    # Unificando todos os arquivos em um só
    result <- do.call(rbind.fill, mydata)

    # Salvando o arquivo reduzido
    write.csv2(result, filestrsave, row.names=FALSE)

    gc()
}

  |======================================================================| 100%

#### <a id="por_cid">Por CIDs</a>
Após gerarmos os arquivos de cada ano, iremos juntar todos os arquivos em um só, porém apenas com as ocorrências referentes a suicídio.

##### X60-X84 Lesões autoprovocadas intencionalmente

-  **X60**: Auto-intoxicação por e exposição, intencional, a analgésicos, antipiréticos e anti-reumáticos, não-opiáceos
-  **X61**: Auto-intoxicação por e exposição, intencional, a drogas anticonvulsivantes [antiepilépticos] sedativos, hipnóticos, antiparkinsonianos e psicotrópicos não classificados em outra parte
-  **X62**: Auto-intoxicação por e exposição, intencional, a narcóticos e psicodislépticos [alucinógenos] não classificados em outra parte
-  **X63**: Auto-intoxicação por e exposição, intencional, a outras substâncias farmacológicas de ação sobre o sistema nervoso autônomo
-  **X64**: Auto-intoxicação por e exposição, intencional, a outras drogas, medicamentos e substâncias biológicas e às não especificadas
-  **X65**: Auto-intoxicação voluntária por álcool
-  **X66**: Auto-intoxicação intencional por solventes orgânicos, hidrocarbonetos halogenados e seus vapores
-  **X67**: Auto-intoxicação intencional por outros gases e vapores
-  **X68**: Auto-intoxicação por e exposição, intencional, a pesticidas
-  **X69**: Auto-intoxicação por e exposição, intencional, a outros produtos químicos e substâncias nocivas não especificadas
-  **X70**: Lesão autoprovocada intencionalmente por enforcamento, estrangulamento e sufocação
-  **X71**: Lesão autoprovocada intencionalmente por afogamento e submersão
-  **X72**: Lesão autoprovocada intencionalmente por disparo de arma de fogo de mão
-  **X73**: Lesão autoprovocada intencionalmente por disparo de espingarda, carabina, ou arma de fogo de maior calibre
-  **X74**: Lesão autoprovocada intencionalmente por disparo de outra arma de fogo e de arma de fogo não especificada
-  **X75**: Lesão autoprovocada intencionalmente por dispositivos explosivos
-  **X76**: Lesão autoprovocada intencionalmente pela fumaça, pelo fogo e por chamas
-  **X77**: Lesão autoprovocada intencionalmente por vapor de água, gases ou objetos quentes
-  **X78**: Lesão autoprovocada intencionalmente por objeto cortante ou penetrante
-  **X79**: Lesão autoprovocada intencionalmente por objeto contundente
-  **X80**: Lesão autoprovocada intencionalmente por precipitação de um lugar elevado
-  **X81**: Lesão autoprovocada intencionalmente por precipitação ou permanência diante de um objeto em movimento
-  **X82**: Lesão autoprovocada intencionalmente por impacto de um veículo a motor
-  **X83**: Lesão autoprovocada intencionalmente por outros meios especificados
-  **X84**: Lesão autoprovocada intencionalmente por meios não especificados

##### Y10-Y19 Intoxicação exógena de intenção indeterminada
- **Y10**: Envenenamento [intoxicação] por e exposição a analgésicos, antipiréticos e anti-reumáticos não-opiáceos, intenção não determinada
- **Y11**: Envenenamento [intoxicação] por e exposição a anticonvulsivantes [antiepilépticos], sedativos, hipnóticos, antiparkinsonianos e psicotrópicos não classificados em outra parte, intenção não determinada
- **Y12**: Envenenamento [intoxicação] por e exposição a narcóticos e a psicodislépticos [alucinógenos] não classificados em outra parte, intenção não determinada
- **Y13**: Envenenamento [intoxicação] por e exposição a outras substâncias farmacológicas de ação sobre o sistema nervoso autônomo, intenção não determinada
- **Y14**: Envenenamento [intoxicação] por e exposição a outras drogas, medicamentos e substâncias biológicas e as não especificadas, intenção não determinada
- **Y15**: Envenenamento [intoxicação] por e exposição ao álcool, intenção não determinada
- **Y16**: Envenenamento [intoxicação] por e exposição a solventes orgânicos e hidrocarbonetos halogenados e seus vapores, intenção não determinada
- **Y17**: Envenenamento [intoxicação] por e exposição a outros gases e vapores, intenção não determinada
- **Y18**: Envenenamento [intoxicação] por e exposição a pesticidas, intenção não determinada
- **Y19**: Envenenamento [intoxicação] por e exposição a outros produtos químicos e substâncias nocivas e aos não especificados, intenção não determinada

##### X87 Sequelas de uma lesão autoprovocada intencionalmente, de agressão ou de um fato cuja intenção é indeterminada
-  **X87.0**: Sequelas de lesões autoprovocadas intencionalmente

Mais detalhes:
- [X60-X84](http://www.datasus.gov.br/cid10/V2008/WebHelp/x60_x84.htm)
- [Y10-Y19](http://www.datasus.gov.br/cid10/V2008/WebHelp/y10_y34.htm)
- [X87.0](http://www.datasus.gov.br/cid10/V2008/WebHelp/y85_y89.htm)

In [3]:
# CIDs
cids = c(paste('X', 60:84, sep=''), 'X87', paste('Y', 10:19, sep=''))

In [4]:
filestrsave <- paste(data_path, paste('suicidios', '.csv', sep=''), sep='/')

# Listando os arquivos
listoffiles <- list.files(anos_path, full.names=TRUE)

# Contador e barra de progresso, respectivamente
i_count <- 0
pb <- txtProgressBar(min=0, max=length(listoffiles), style=3)

# Abrindo os arquivos
mydata <- lapply(listoffiles, function (file) {
    # Pegamos o arquivo, transformamos ele em list
    data_csv <- read.csv(file, header=TRUE, sep=';')
    # Pegamos apenas as ocorrências de CIDs que queremos
    data_csv <- subset(data_csv, grepl(paste(cids, collapse='|'), CAUSABAS), select=cols_name)
    # Avançamos no contador, ou seja, na barra de progresso
    i_count <<- i_count + 1
    setTxtProgressBar(pb, i_count)

    return(data_csv)
})

# Unificando todos os arquivos em um só
result <- do.call(rbind.fill, mydata)

# Salvando o arquivo reduzido
write.csv2(result, filestrsave, row.names=FALSE)

gc()

  |======================================================================| 100%

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,781716,41.8,3763080,201.0,4589385,245.1
Vcells,5729011,43.8,234384696,1788.3,292916148,2234.8
